In [156]:
# coding:utf-8
# Textual analysis through the open web
from __future__ import print_function
import io
import os
import bs4
import sys
import json
import time
import urllib
import requests
import wikipedia

In [204]:
def read_json(path):
    data = ''
    with io.open(path, 'r', encoding='utf-8') as f:
        data = json.loads(f.read())
        print(__message(u'Loaded {}'.format(path)))
    return data
    
def modify_game_meta(data):
    for k, v in data.items():
        data[k].pop('Last', None)
        data[k]['Year'] = int(data[k]['First'][:4])
        data[k].pop('First', None)
        data[k].pop('Title', None)
        data[k]['Summary'] = scrape_wiki(k)
        
#         # Scraping Duck Duck Go
#         print(__message('Scraping DuckDuckGo for a list of URLs...'))
#         new_links = scrape_duckduckgo(k, data[k]['Developer'])
#         data[k]['Links'] = new_links | set(data[k]['Links'])

    print(__message('Done'))
    return data

def scrape_wiki(title):
    try:
        searchstring = title
        summary = wikipedia.summary(searchstring)
        print(__success(searchstring))
    except wikipedia.DisambiguationError:
        try:
            searchstring = u'{} (video game)'.format(title).replace(u' ', u'_')
            summary = wikipedia.page(searchstring, auto_suggest=False).summary
            print(__success(searchstring))
        except wikipedia.DisambiguationError:
            searchstring = u'{} (Unix video game)'.format(title).replace(u' ', u'_')
            summary = wikipedia.page(searchstring, auto_suggest=False).summary
            print(__success(searchstring))
        except wikipedia.PageError:
            summary = 'No summary found on Wikipedia.'
            print(__warning(u'Wikipedia cannot find "{}"'.format(searchstring)))
    except wikipedia.PageError:
        try:
            summary = wikipedia.page(searchstring, auto_suggest=False).summary
            print(__success(searchstring))
        except wikipedia.PageError:
            summary = 'No summary found on Wikipedia.'
            print(__warning(u'Search term "{}" returned nothing'.format(searchstring)))
    return summary

def scrape_duckduckgo(keywords, developer=""):
    searchstring = u'"{}" {} {}'.format(keywords, developer, u'interview game')
    response = requests.get(u'http://duckduckgo.com/html/?q={}'.format(
                urllib.quote(searchstring.encode('utf-8'))),
                timeout=(10, 15)
             )
    print(__message(u'DDG: {}'.format(searchstring)))
    soup = bs4.BeautifulSoup(response.text)
    links = []
    for node in soup.select('div.web-result'):
        if 'web-result-sponsored' in node['class']:
            continue
        try:
            links.append(node.select('a.large')[0].get('href'))
        except Exception as e:
            print(__failure(e))
            pass
    if links:
        print(__success(u'DDG: {}'.format(searchstring)))
        print('\t\t\n'.join(links))
    return links

def scrape_url(url):
    try:
        response = requests.get(url, timeout=(10, 15))
        return response.text
    except Exception as e:
        print(__failure(e))

def clean_url_data(links, data):
    links = set(links)
    for link in data.keys():
        if link not in links:
            data.pop(link, None)
    return data
    
def save_json(path, data):
    with io.open(path, 'w', encoding='utf-8') as f:
        try:
            output = json.dumps(data, indent=2, ensure_ascii=False)
            f.write(output)
        except UnicodeEncodeError:
            f.write(output.encode('utf-8'))
    print(__message(u'Written to {}'.format(path)))
        
def __success(text):
    return u'  (SUCC) {}'.format(text).encode('utf-8')
    
def __failure(text):
    return u'!!FAIL!! {}'.format(text).encode('utf-8')
    
def __warning(text):
    return u'??WARN?? {}'.format(text).encode('utf-8')
    
def __message(text):
    return u'   |MSG| {}'.format(text).encode('utf-8')

In [ ]:
cwd = os.getcwd()

game_meta = read_json(os.path.join(cwd, 'data', 'game-sources.json'))

for game, meta in game_meta.items():
    game_meta[game]['Links'] += scrape_duckduckgo(game, game_meta[game]['Developer'])
    game_meta[game]['Links'] = list(set(game_meta[game]['Links']))
    save_json(os.path.join(cwd, 'data', 'game-sources.json'), game_meta)

   |MSG| Loaded /Users/spaxe/dev/roguelike-universe/data/game-sources.json
   |MSG| DDG: "Dungeon" Brian Sawyer interview game
  (SUCC) DDG: "Dungeon" Brian Sawyer interview game
http://www.gatheryourparty.com/2012/10/10/interview-josh-sawyer/		
http://www.gamewatcher.com/rpg		
http://fixsattamatka.blogspot.com/		
http://www.pcgamesn.com/darkest-dungeon		
http://www.gamebanshee.com/46-404.html		
http://forums.penny-arcade.com/discussion/124626/dungeon-siege-3-obsidians-action-rpg-nsf-56k/p13		
http://rpgcodex.net/		
https://en.wikipedia.org/wiki/Fritzl_case		
http://www.gamesindustry.biz/articles/2013-04-09-man-of-many-parts-chris-avellones-busy-schedule		
http://www.how-to-diy.org/MY2n6FiXu80gpa/Matt-Chat-91:-The-Fall-of-Interplay-with-Brian-Fargo.html		
http://www.rockpapershotgun.com/2013/01/09/planescape-torment-sequel/		
http://archive.arstechnica.com/etc/games/2002/gars-03062002.html		
http://kristinandcory.com/family_interview_2.html		
http://www.bethblog.com/2007/09/11/daggerfa